In [1]:
import datasets
import random
import re

from common.utils import get_refs
from common.consts import DS_UPLOAD_PATH, DS_SAVE_PATH

/home/tdolega/miniconda/envs/p311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original = datasets.load_dataset(DS_UPLOAD_PATH)
translation = datasets.load_from_disk(DS_SAVE_PATH + "-pl")
translation

DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 8348
    })
    test: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 128
    })
})

In [3]:
def print_random(ds, n=2, seed=50):
    examples = ds["train"].shuffle(seed=seed).select(range(n))
    for example in examples:
        print(example["prompt"])
        print()
        print(example["answer"])
        print()
        print()


print_random(translation)

Dokument [1](Tytuł: David Lloyd (nadawca)): David Lloyd jest angielskim prezenterem radiowym i byłym dyrektorem radiowym,
Dokument [2](Tytuł: Gary King (radio)): Gary King (ur. 4 sierpnia 1963) to angielski prezenter radiowy, który pracował dla Magic 105.4 Capital FM, BBC Radio 1 i Jazz FM (Wielka Brytania). Był pierwszym prezenterem na Atlantyku 252, uruchamiając stację w dniu 1 września 1989 r.
Dokument [3](Tytuł: Alan Robson): Alan Robson MBE (urodzony 1 października 1955) to angielski prezenter radiowy, który prowadzi szeroko słuchany talk show „Alan Robson's Night Owls” w Metro Radio i TFM Radio w północno-wschodniej Anglii. Twierdzi, że w listopadzie 2000 roku uzyskał tytuł magistra egzorcyzmu i spirytualizmu na Birkbeck na Uniwersytecie Londyńskim.
Dokument [4](Tytuł: Simon Mayo): Simon Andrew Hicks Mayo (urodzony 21 września 1958) to angielski prezenter radiowy, który pracuje dla BBC Radio od 1981. Mayo jest prezenterem „Simon Mayo Drivetime” w BBC Radio 2, które robi to od 201

In [4]:
def remove_translations_with_bad_citations(original, translation, question_prefix="\nPytanie:"):
    filtered_dataset = datasets.DatasetDict()
    for split in ["train", "test"]:
        original_split = original[split]
        translation_split = translation[split]
        filtered = []
        assert len(original_split) == len(translation_split)
        for i, (original_examples, translated_examples) in enumerate(zip(original_split, translation_split)):
            original_refs = get_refs(original_examples["answer"])
            translated_refs = get_refs(translated_examples["answer"])
            if set(original_refs) != set(translated_refs):
                continue
            original_refs_passage = get_refs(original_examples["prompt"])
            translated_refs_passage = get_refs(translated_examples["prompt"])
            if set(original_refs_passage) != set(translated_refs_passage):
                continue
            if original_examples["prompt"].count("\n") != translated_examples["prompt"].count("\n"):
                continue
            if question_prefix not in translated_examples["prompt"]:
                continue
            filtered.append(i)
        print(f"Filtered {len(original_split) - len(filtered)} ({(len(original_split) - len(filtered))/len(original_split):.2%}) examples from {split}")
        filtered_dataset[split] = translation_split.select(filtered)
    return filtered_dataset


filtered_pl = remove_translations_with_bad_citations(original, translation)
filtered_pl

Filtered 499 (5.98%) examples from train
Filtered 6 (4.69%) examples from test


DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 7849
    })
    test: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 122
    })
})

In [5]:
def fixup_translation(row):
    row["answer"] = row["answer"].replace("] [", "][")
    row["prompt"] = row["prompt"].replace("Document [", "Dokument [")
    return row


fixed_pl = filtered_pl.map(fixup_translation)

In [6]:
DOC_TEMPLATE = "Dokument [{ref}]"
ANS_TEMPLATE = "[{ref}]"


def remove_docs(prompt, refs):
    for ref in refs:
        if ref > 100:
            print(f"Skipping ref {ref}")
            continue
        needle = DOC_TEMPLATE.format(ref=f"{ref+1}")
        needle_idx = prompt.find(needle)
        next_newline_idx = prompt.find("\n", needle_idx)
        if next_newline_idx == -1:
            next_newline_idx = len(prompt)
            print(f"needle: {needle}\n\nneedle_idx: {needle_idx}\n\nprompt: {prompt}")
            0 / 0
        prompt = prompt[:needle_idx] + prompt[next_newline_idx + 1 :]

    new_prompt_refs = get_refs(prompt)
    for i, ref in enumerate(new_prompt_refs):
        prompt = prompt.replace(DOC_TEMPLATE.format(ref=ref + 1), DOC_TEMPLATE.format(ref=i + 1))

    return prompt


def add_mapped_samples(ds, amount, fn):
    random.seed(9)
    new_ds = datasets.DatasetDict()
    for split_name in ["train", "test"]:
        split = ds[split_name]
        selected = split.shuffle(seed=8)
        selected = selected.select(range(int(len(split) * amount)))
        selected = selected.map(fn)
        split = datasets.concatenate_datasets([split, selected])
        split = split.shuffle(seed=5)
        new_ds[split_name] = split
    return new_ds


def renumerate_answer_refs(answer, removed_refs):
    refs = get_refs(answer)
    for i, ref in enumerate(refs):
        assert ref not in removed_refs
        removed_below = len([r for r in removed_refs if r < ref])
        answer = answer.replace(ANS_TEMPLATE.format(ref=ref + 1), ANS_TEMPLATE.format(ref=ref - removed_below + 1))
    return answer


def change_to_reduced(row):
    refs = get_refs(row["answer"])
    prompt_refs = get_refs(row["prompt"])
    non_refs = [ref for ref in prompt_refs if ref not in refs]
    if len(non_refs) == 0:
        return row
    refs_to_remove = random.sample(non_refs, random.randint(1, len(non_refs)))

    return {
        "prompt": remove_docs(row["prompt"], refs_to_remove),
        "answer": renumerate_answer_refs(row["answer"], refs_to_remove),
    }


with_reduced_pl = add_mapped_samples(fixed_pl, 1 / 6, change_to_reduced)
with_reduced_pl

DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 9157
    })
    test: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 142
    })
})

In [7]:
NEGATIVE_ANSWER = "Przepraszam, nie potrafię odpowiedzieć na to pytanie."


def change_to_negative(row):
    refs = get_refs(row["answer"])
    return {"prompt": remove_docs(row["prompt"], refs), "answer": NEGATIVE_ANSWER}


with_neg_pl = add_mapped_samples(with_reduced_pl, 1 / 8, change_to_negative)
with_neg_pl

DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 10301
    })
    test: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 159
    })
})

In [8]:
TITLE_KEY = "Tytuł"
TITLE_REMOVE_REGEX = re.compile(r"\]\s?\({}:.*\):".format(TITLE_KEY), re.IGNORECASE)
TITLE_REMOVE_REPLACE = "]:"


def remove_title(row):
    row["prompt"] = TITLE_REMOVE_REGEX.sub(TITLE_REMOVE_REPLACE, row["prompt"])
    return row


without_title_pl = with_neg_pl.map(remove_title)

In [9]:
print_random(without_title_pl, 10)

Dokument [1]: Whaley House to rezydencja w stylu greckiego odrodzenia z 1857 r., zabytek historyczny Kalifornii i muzeum zlokalizowane na Starym Mieście w San Diego w Kalifornii. Obecnie opiekuje się nim Organizacja Save Our Heritage (SOHO). W latach sześćdziesiątych XX wieku Whaley House został wyznaczony przez Departament Handlu Stanów Zjednoczonych jako oficjalny nawiedzony dom.
Dokument [2]: Dom Roberta Henne'a znajduje się w dzielnicy West End w Davenport w stanie Iowa, Stany Zjednoczone. Ta rezydencja w stylu greckiego odrodzenia została zbudowana dla Roberta i Henrietty Henne w 1874 roku. To on obsługiwał stoisko z cygarami na poczcie. Kontynuowała działalność po jego śmierci w 1885 roku. Dom utrzymany był w popularnym XIX-wiecznym stylu w Davenport, który miał pewne unikalne cechy. Okulus szczytowy znajduje się z boku domu, a nie z przodu. Z przodu para okien o okrągłym łuku. Okna wychodzące na przód domu mają zwornikowe nadproża, które opadają do małych formowanych bloków naro

In [12]:
final_ds = without_title_pl
final_ds.save_to_disk(DS_SAVE_PATH + "-pl" + "-v2")
final_ds.push_to_hub(DS_UPLOAD_PATH + "_pl")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/tdolega/rag-tge_finetuning-dataset_pl/commit/1b45821342e74067885868e98315374db694429e', commit_message='Upload dataset', commit_description='', oid='1b45821342e74067885868e98315374db694429e', pr_url=None, pr_revision=None, pr_num=None)